In [19]:
import pandas as pd
from sodapy import Socrata

pd.set_option('display.width', 1505)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

# to read from local static copy
df = pd.read_csv("~/Downloads/Food_Inspections.csv")
# to pull most recent version and read from the SODA API
client = Socrata("data.cityofchicago.org", None)
results = client.get("4ijn-s7e5", limit=2000)
results_df = pd.DataFrame.from_records(results)


#Drop all rows that arent restauratnts, and those for which the facility type or the zip code are NaN
results_df = results_df.dropna(subset=['facility_type', 'zip'])
results_df['facility_type'] = results_df['facility_type'].str.lower()
results_df['results'] = results_df['results'].str.lower()
restaurants_only_bool = results_df['facility_type'].str.contains("rest")



#Drop all rows where the license is not > 0
valid_license_bool = (results_df['license_'] != 0)
failed_inspection_bool = (results_df['results'] == 'fail')
passed_inspection_bool = (results_df['results'] == 'pass')
restaurants_only_df = results_df[restaurants_only_bool & valid_license_bool & (failed_inspection_bool | passed_inspection_bool)]

#Drop the columns that we dont really care about
restaurants_only_df = restaurants_only_df.drop(columns=['inspection_id', 'aka_name', 'risk',
 'address', 'city', 'state','zip', 'inspection_date', 'inspection_type', 'location', 
 ':@computed_region_awaf_s7ux', ':@computed_region_6mkv_f3dw', ':@computed_region_vrxf_vc4k',
 ':@computed_region_bdys_3d7i' ,':@computed_region_43wa_7qmu'])


print(restaurants_only_df.head())



['inspection_id' 'dba_name' 'aka_name' 'license_' 'facility_type' 'risk'
 'address' 'city' 'state' 'zip' 'inspection_date' 'inspection_type'
 'results' 'latitude' 'longitude' 'location' ':@computed_region_awaf_s7ux'
 ':@computed_region_6mkv_f3dw' ':@computed_region_vrxf_vc4k'
 ':@computed_region_bdys_3d7i' ':@computed_region_43wa_7qmu' 'violations']
